In [6]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import matplotlib.pyplot as plt
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from tqdm import tqdm
import math
import xgboost
import xgboost as xgb
import pickle
import duckdb

In [7]:
from binding_prediction.const import WHOLE_MOLECULE_COLUMN, TARGET_COLUMN

from binding_prediction.datasets.xgboost_iterator import SmilesIterator

from binding_prediction.evaluation.kaggle_submission_creation import get_submission_test_predictions_for_xgboost_model


# EDA

## Get train dataset info

In [8]:
train_path = 'data/train.parquet'
test_path = 'data/test.parquet'

In [ ]:
parquet_file = pq.ParquetFile('data/train.parquet')

In [ ]:
parquet_file.metadata

In [ ]:
row_group_0 = parquet_file.read_row_group(0)
row_group_1 = parquet_file.read_row_group(1)

combined_table = pa.concat_tables([row_group_0, row_group_1])
pq.write_table(row_group_0, 'data/row_group_0.parquet')
pq.write_table(combined_table, 'data/two_row_groups.parquet')


In [ ]:
for i in range(parquet_file.metadata.num_row_groups):
    row_group_stats = parquet_file.metadata.row_group(i).column(0).statistics
    print(f"row group: {i}, num of rows: {row_group_stats.num_values}")

### Create subsample of train dataset to experiment with it

In [ ]:
# https://www.kaggle.com/code/andrewdblevins/leash-tutorial-ecfps-and-random-forest
con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 3000000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 1589906)""").arrow()
pq.write_table(df, 'data/subsampled_5M_train.parquet')

con.close()

In [ ]:
row_group_df = parquet_file.read_row_group(0).to_pandas()
row_group_df.to_csv('data/row_group_0.csv')

In [ ]:
row_group_df.info()

### Get statistics of the target column

In [ ]:
negative_count = 0
positive_count = 0
for i in range(parquet_file.metadata.num_row_groups):
    row_group_df = parquet_file.read_row_group(i).to_pandas()
    negative_count += row_group_df[TARGET_COLUMN].value_counts()[0]
    positive_count += row_group_df[TARGET_COLUMN].value_counts()[1]
    print(f"row group: {i}, negative count: {negative_count}, positive count: {positive_count}")

In [ ]:
_ = plt.bar(['log negative', 'log positive'], [np.log(negative_count), np.log(positive_count)])
_ = plt.title('Target column distribution')

### Draw couple of molecules

In [ ]:
row_group_df = parquet_file.read_row_group(0).to_pandas()

In [ ]:
row_group_df[WHOLE_MOLECULE_COLUMN].head()

In [ ]:
smiles = []
targets = []
for i in tqdm(range(parquet_file.metadata.num_row_groups)):
    row_group_df = parquet_file.read_row_group(i).to_pandas()
    negative_sample = row_group_df[row_group_df[TARGET_COLUMN] == 0].sample(2)
    positive_sample = row_group_df[row_group_df[TARGET_COLUMN] == 1].sample(2)
    subsample = pd.concat([negative_sample, positive_sample])
    smiles.extend(subsample[WHOLE_MOLECULE_COLUMN].values)
    targets.extend(subsample[TARGET_COLUMN].values)

In [ ]:
num_mols_to_draw = 100
random_indices = np.random.choice(len(smiles), num_mols_to_draw, replace=False)
grid_size = math.ceil(np.sqrt(num_mols_to_draw))
fig, axs = plt.subplots(grid_size, grid_size, figsize=(20, 20))
for i in tqdm(range(num_mols_to_draw)):
    smile = smiles[random_indices[i]]
    target = targets[random_indices[i]]
    mol = Chem.MolFromSmiles(smile)
    ax = axs[i // grid_size, i % grid_size]
    ax.imshow(Draw.MolToImage(mol))
    ax.set_title(f'target: {target}')
ax.axis('off')

# XGBoost baseline

In [ ]:
print('Train validation split')
train_file_path = 'data/row_group_0.parquet'
rng = np.random.default_rng(seed=42)

train_val_pq = pq.ParquetFile(train_file_path)
train_indices = rng.choice(train_val_pq.metadata.num_rows, int(0.8 * train_val_pq.metadata.num_rows), replace=False)
val_indices = np.setdiff1d(np.arange(train_val_pq.metadata.num_rows), train_indices)

print('Creating datasets')
train_dataset = SmilesIterator(train_file_path, indicies=train_indices, radius=3, test_set=False)
val_dataset = SmilesIterator(train_file_path, indicies=val_indices, radius=3, test_set=False)
test_dataset = SmilesIterator('data/test.parquet', shuffle=False, radius=3, test_set=True)

In [ ]:
train_Xy = xgboost.DMatrix(train_dataset)

In [ ]:
val_Xy = xgboost.DMatrix(val_dataset)

In [ ]:
test_Xy = xgboost.DMatrix(test_dataset)

In [ ]:
print('Load model')
model_path = 'logs/2024-04-25_22-04-59/model.pkl'
with open(model_path, 'rb') as file:
    model = pickle.load(file)
    

In [ ]:
print('Predicting')
get_submission_test_predictions(test_dataset, test_Xy, model, 'logs/2024-04-25_22-04-59')

In [ ]:
print('Creating model')
params = {
    'max_depth': 10,
    'objective': 'binary:logistic',
    # 'nthread': 4,
    'eval_metric': 'auc',
    'verbosity': 2
}


num_rounds = 10  # equivalent to the number of estimators

eval_list = [(train_Xy, 'train'), (val_Xy, 'eval')]  # evaluation set for monitoring

In [ ]:
model = xgb.train(params, train_Xy, num_rounds, evals=eval_list, verbose_eval=100000)

In [ ]:
# Save the model using pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)